In [ ]:
!pip install line_profiler
!pip install memory_profiler

In [ ]:
%load_ext memory_profiler
%load_ext line_profiler

%reload_ext memory_profiler
%reload_ext line_profiler

In [ ]:
import oracledb

host="host"
port='port'
sname = 'sname'

user = "username"
password = "password"

cs =  host+":"+port+"/"+sname
conn = oracledb.connect(user=un, password=pw, dsn=cs)

In [ ]:
# Establish the connection
try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    cursor = connection.cursor()
    print("Connection successful!")
except oracledb.DatabaseError as e:
    print("There was an error connecting to the database:", e)

#### Create Tables

**TABLE CREATION TABLES - ORACLE**

The below table generation queries can be copied and pasted into a query window in oracle and run accordingly in order to create the necessary table structures.

***NB: The short query that comes after - should be run after all the data have been imported to create a index between the handplayers and hands tables.***

In [ ]:
CREATE TABLE PLAYERS(
    PID int primary key,
    PName varchar(100),
    PAge int,
    PCountry varchar(100)
);

CREATE TABLE TOURNAMENTS(
    TID int primary key,
    TName varchar(255),
    TLoc varchar(100),
    TDate date,
    TBuyin FLOAT
);

CREATE TABLE TOURNAMENTSWINNERS(
    TWID int primary key,
    TID int NOT NULL,
    Payout int,
    PID int NOT NULL,
    FOREIGN KEY (TID) REFERENCES TOURNAMENTS(TID),
    FOREIGN KEY (PID) REFERENCES PLAYERS(PID)
);

CREATE TABLE ROUNDS(
    RoundID int primary key,
    TID int NOT NULL,
    RoundNum int,
    FOREIGN KEY (TID) REFERENCES TOURNAMENTS(TID)
);

CREATE TABLE GAMES(
    GameID int primary key,
    RoundID int NOT NULL,
    GameNum int,
    FOREIGN KEY (RoundID) REFERENCES ROUNDS(RoundID)
);

CREATE TABLE BOARD(
    BoardID int primary key,
    FC1 varchar(6),
    FC2 varchar(6),
    FC3 varchar(6),
    TC1 varchar(6),
    RC1 varchar(6)
);

CREATE TABLE HANDS(
    HandID int primary key,
    HandNum int,
    GameID int NOT NULL,
    Dealer int NOT NULL,
    BoardID int NOT NULL,
    TotalPot int,
    FOREIGN KEY (GameID) REFERENCES GAMES(GameID),
    FOREIGN KEY (BoardID) REFERENCES BOARD(BoardID)
);

CREATE TABLE HANDPLAYERS(
    HPID int primary key,
    HandID int NOT NULL,
    PID int NOT NULL,
    POS int NOT NULL,
    FOREIGN KEY (HandID) REFERENCES HANDS(HandID),
    FOREIGN KEY (PID) REFERENCES PLAYERS(PID)
);


CREATE TABLE ELIMINATED(
    EID int primary key,
    HPID int NOT NULL,
    FOREIGN KEY (HPID) REFERENCES HANDPLAYERS(HPID)
);

CREATE TABLE CONTINUING(
    GCID int primary key,
    HPID int NOT NULL,
    FOREIGN KEY (HPID) REFERENCES HANDPLAYERS(HPID)
);

CREATE TABLE HOLE(
    HID int primary key,
    HPID int NOT NULL,
    STAGE varchar(50),
    Card1 varchar(6),
    Card2 varchar(6),
    ACT varchar(50),
    ChipAmt int,
    FOREIGN KEY (HPID) REFERENCES HANDPLAYERS(HPID)
);

CREATE TABLE CARDS(
    CID int primary key,
    HPID int NOT NULL,
    STAGE varchar(50),
    HANDRANK varchar(50),
    ACT varchar(50),
    AllIn int,
    ChipAmt int,
    FOREIGN KEY (HPID) REFERENCES HANDPLAYERS(HPID)
);

CREATE TABLE HANDWINNERS(
    HWID int primary key,
    HPID int NOT NULL,
    Pot int,
    FOREIGN KEY (HPID) REFERENCES HANDPLAYERS(HPID)
);

#### Add Dealer Foreign Key reference

In [ ]:
ALTER TABLE HANDS
ADD CONSTRAINT FK_Dealer
FOREIGN KEY (Dealer) REFERENCES HANDPLAYERS(HPID);

#### Add Players

In [ ]:
def pushPlayers():
    q = "INSERT INTO PLAYERS (PID, PName, PAge, PCountry) VALUES (:id, :name, :age, :country)"
    c.executemany(q, [(key, value['PName'], value['PAge'],value['PCountry'] ) for key, value in players.items()])

In [ ]:
import csv

players={}

# Open the CSV file and read its contents
with open('PLAYERS_INFO_1024.csv', 'r') as file:
    line = csv.reader(file, delimiter=',')
    line_count = 0
    for row in line:
        #print(row)
        if line_count != 0:

            #names with apostrophe
            parts = row[1].split(" ")
            firstname = parts[0]
            lastname = " ".join(parts[1:])  # Join the rest of the parts as the last name

            # If the last name contains an apostrophe, handle it properly
            if "'" in lastname:
                lastname = lastname.replace("'", "''")
            name = firstname +" "+ lastname
            play = {
                "PName":name,
                "PAge": row[2],
                "PCountry": row[3],
            }
            players[int(row[0])] = play


        line_count += 1

pushPlayers()
conn.commit()

In [ ]:
file_path = "data/Players.csv"
parse_and_process_players(file_path, connection)

### Parse Data into Oracle DB

In [ ]:
file = "data/games/pokergames1.txt"

**Tournaments Info**

In [ ]:
def pushBulkImport(tournaments,rounds,games,hands,hands_players,hand_board,holeC,cards,hand_winners,game_elim,game_con,tour_winners):
    import datetime
date_obj = datetime.datetime(2023, 12, 25)

    # Format the date as a string in the desired format
    formatted_date = date_obj.strftime('%Y-%m-%d %H:%M:%S')

    def pushTournaments(t):
        #print("a")
        q = "INSERT INTO TOURNAMENTS(TID, TName, TLoc, TDate, TBuyin) VALUES (:tid, :tname, :tloc, TO_DATE(:tdate, 'YYYY-MM-DD HH24:MI:SS'), :tbuyin)"
        data = [{'tid': key, 'tname': value['TName'], 'tloc': value['TLoc'], 'tdate': value['TDate'], 'tbuyin': value['TBuyin']} for key, value in t.items()]
        c.executemany(q, data)
        conn.commit()


    def pushTournWinners(tw):
        #print("b")
        q = f"INSERT INTO TOURNAMENTSWINNERS(TWID, TID, PAYOUT, PID) VALUES(:TWID, :TID, :PAYOUT, :PID )"
        data = [{'TWID': key, 'TID': value['TID'], 'PAYOUT': value['PAYOUT'], 'PID': value['PID']} for key, value in tw.items()]
        c.executemany(q, data)
        conn.commit()


    def pushRounds(r):
        #print("c")
        q = "INSERT INTO ROUNDS (RoundID, TID, RoundNum) VALUES (:roundid, :tid, :roundnum)"
        c.executemany(q, [(key, value['TID'], value['RoundNum']) for key, value in r.items()])
        conn.commit()


    def pushGames(g):
        #print("d")
        q = "INSERT INTO GAMES (GameID, RoundID, GameNum) VALUES (:gameid, :rid, :gamenum)"
        c.executemany(q, [(key, value['RoundID'], value['GameNum']) for key, value in g.items()])
        conn.commit()


    def pushHands(h):
        #print("e")
        q = f"INSERT INTO HANDS(HandID, HandNum, GameID, Dealer, BoardID, TotalPot) " \
            f"VALUES(:handid, :handnum, :gid, :dealer, :board, :potamt)"

        c.executemany(q, [(key, value['HandNum'], value['GameID'],value['Dealer_HPID'],value['BoardID'],value['PotTotal'] ) for key, value in h.items()])
        conn.commit()


    def pushHandPlayers(hp):
        #print("f")
        q = f"INSERT INTO HANDPLAYERS(HPID, HANDID, PID, POS) VALUES(:hpid, :handid, :pid, :position )"
        c.executemany(q, [(key, value['HANDID'], value['PID'],value['POS'] ) for key, value in hp.items()])
        conn.commit()


    def pushHole(hl):
        #print("g")
        q = f"INSERT INTO HOLE(HID, HPID, STAGE, CARD1, CARD2, ACT, CHIPAMT)" \
            f"VALUES(:hid, :hpid, :stage, :card1, :card2, :act, :chipamt   )"
        c.executemany(q, [(key, value['HPID'], value['STAGE'],value['CARD1'],value['CARD2'],value['ACT'],value['CHIPAMT'] ) for key, value in hl.items()])
        conn.commit()


    def pushCards(k):
        #print("h")
        q = f"INSERT INTO CARDS(CID, HPID, STAGE, HANDRANK, ACT, ALLIN, CHIPAMT) VALUES (:cid, :hpid, :stage, :handrank, :act, :allin, :chipamt)"
        c.executemany(q, [(key, value['HPID'], value['STAGE'],value['HANDRANK'],value['ACT'],value['AllIn'],value['CHIPAMT'] ) for key, value in  k.items()])
        conn.commit()


    def pushBoard(b):
        #print("i")
        q = f"INSERT INTO BOARD(BOARDID, FC1,FC2,FC3,TC1,RC1) VALUES(:id, :fc1, :fc2,  :fc3, :tc, :rc)"
        c.executemany(q, [(key, value['FC1'], value['FC2'],value['FC3'],value['TC'],value['RC']) for key, value in b.items()])
        conn.commit()

    def pushHandWinners(hw):
        #print("j")
        q = f"INSERT INTO HANDWINNERS(HWID, HPID, POT) VALUES(:hwid, :hpid,  :pot )"
        c.executemany(q, [(key, value['HPID'],int(value['POT']) ) for key, value in hw.items()])
        conn.commit()


    def pushGameEliminated(ge):
        #print("k")
        q = f"INSERT INTO ELIMINATED(EID, HPID) VALUES(:id, :hpid)"
        c.executemany(q, [(key, value['HPID'] ) for key, value in ge.items()])
        conn.commit()


    def pushGameContinue(gc):
        #print("l")
        q = f"INSERT INTO CONTINUING(GCID, HPID) VALUES(:id, :hpid)"
        c.executemany(q, [(key, value['HPID']) for key, value in gc.items()])
        conn.commit()


    #call functions to push data to Oracle
    pushTournaments(tournaments)
    pushTournWinners(tour_winners)
    pushRounds(rounds)
    pushGames(games)
    pushBoard(hand_board)
    pushHands(hands)
    pushHandPlayers(hands_players)
    pushGameEliminated(game_elim)
    pushGameContinue(game_con)
    pushHole(holeC)
    pushHandWinners(hand_winners)
    pushCards(cards)


#### Parse Game Files

In [ ]:
#PARSING FILES
def ParseAndImport():
    #INITALIZE
    #***************************************************
    #DATASETS
    tournaments = {}
    rounds = {}
    games = {}
    hands = {}

    hands_players={}
    hand_board={}

    holeC={}
    cards={}
    card_ref={}
    hole_ref={}

    hand_winners={}
    winner={}
    game_elim={}
    game_con={}

    tour_winners={}

    hp_id={} # holds all Hand_Player IDs for players in a hand

    record_hand = False # Flag to indicate when to start recording PL values
    record_action = False

    #***************************************************
    #ID counters
    count=0 #player's position in hand
    acount=0 #action id
    hcount=0 #hand id

    hpcount=0 #hand player id
    hpid=None
    ccount=0 #card id
    bcount=0 #board id
    hwcount=0 #hand winner id
    ecount=0 #eliminated id
    concount=0 #game continuing id
    twcount=0 #tornamentswinnersid

    rcount=0 #roundid
    gcount=0 #gameid
    holecount=0 #holeid

    handpot_total=0
    allin=0

    def reset():
        tournaments.clear()
        rounds.clear()
        games.clear()
        hands.clear()

        hands_players.clear()
        hand_board.clear()

        holeC.clear()
        cards.clear()
        card_ref.clear()
        hole_ref.clear()

        hand_winners.clear()
        winner.clear()
        game_elim.clear()
        game_con.clear()

        tour_winners.clear()

        hp_id.clear() # holds all Hand_Player IDs for players in a hand

        record_hand = False # Flag to indicate when to start recording PL values
        record_action = False

        allin=0
        handpot_total=0
    #***************************************************
    for f in files:
        # Read data from file
        with open(f, 'r') as file:
            lines = file.readlines()
            for line in lines:
                line = line.strip()

                if line.startswith("TOURNAMENTEND"):

                    # Assuming tid is the unique identifier for tournaments
                    torn_info = {
                        #"TID": tid,
                        "TName": tname,
                        "TLoc": tloc,
                        "TDate": tdate,
                        "TBuyin": tbuyin
                    }

                    # Update tournaments with torn_info as a key-value pair
                    tournaments[tid] = torn_info


                    pushBulkImport(tournaments,
                                   rounds,
                                   games,
                                   hands,
                                   hands_players,
                                   hand_board,
                                   holeC,
                                   cards,
                                   hand_winners,
                                   game_elim,
                                   game_con,
                                   tour_winners
                                  ) #push data to database
                    reset()
                    print("end",tid)
                # TORNAMENT INFO
                if line.startswith("TOURNAMENT:"):
                    tid = int(line.split(":")[1].strip())
                    print("start",tid)

                if line.startswith("NAME:"):
                    tname = line.split(":")[1].strip()

                if line.startswith("LOCATION:"):
                    tloc = line.split(":")[1].strip()

                if line.startswith("DATE:"):
                    tdate= line.split(":")[1].strip()

                if line.startswith("BUYIN:"):
                    tbuyin = float(line.split(":")[1].strip())


                # HAND INFO
                elif line.startswith("ROUND:"):
                    rid = int( line.split(":")[1])
                    rcount+= 1
                    round_info = {#"RoundID": rcount,
                                  "TID": tid,
                                  "RoundNum":rid}
                    rounds[rcount] = round_info

                elif line.startswith("GAME:"):
                    gid = int(line.split(":")[1])
                    gcount+= 1
                    game_info = {#"GameID": gcount,
                                 "RoundID": rcount,
                                 "GameNum":gid}

                    games[gcount]= game_info

                elif line.startswith("HAND:"): #HAND_INFO
                    hp_id.clear()
                    hcount+=1
                    hid = int(line.split(":")[1])
                    record_hand = True

                elif line.startswith("HANDEND:"):
                    dealer = next((key for key, val in hp_id.items() if val['PID'] == deal), None)
                    hand_info = {#"HandID": hcount,
                                 "HandNum": hid,
                                 "GameID": gcount,
                                 "Dealer_HPID": dealer,
                                 "BoardID": bcount,
                                 "PotTotal": handpot_total}

                    hands[hcount]=hand_info
                    record_hand = False
                    count=0
                    handpot_total=0



                elif record_hand and line.startswith("PL:"):
                    count+=1
                    hpcount+=1
                    pid, buyin = line.split(":")[1].split(",")

                    i={#"HPID": hpcount,
                       "HANDID": hcount,
                       "PID": int(pid),
                       "POS": count}

                    h = {#"HPID": hpcount,
                         "PID":int(pid)}

                    hands_players[hpcount]=i
                    hp_id[hpcount]= h

                elif record_hand and line.startswith("DEALER:"):
                    bcount+=1
                    deal = int(line.split(":")[1])

                elif record_hand and line.startswith("PREFLOP:"):
                    stage = line.split(":")[0]
                    record_action = True
                    hole_ref.clear()

                elif record_action and line.startswith("HOLE:"):
                    hole = line.split(":")[1].split(",")
                    hpid = next((key for key, val in hp_id.items() if val['PID'] == int(hole[0])), None)

                    k={
                       "PID": int(hole[0]),
                       "card1": hole[1],
                        "card2": hole[2],
                        }
                    hole_ref[hpid]=k

                elif record_action and line.startswith("ACT:"):
                    pf_act = line.split(":")[1].split(",")
                    handpot_total += int(pf_act[2])

                    #get players handplayer id
                    hpid = next((key for key, val in hp_id.items() if val['PID'] == int(pf_act[0])), None)
                    card_vals = next((val for key, val in hole_ref.items() if key == hpid), None)

                    #********************ACT

                    if stage == "PREFLOP":   #hold
                        holecount+=1
                        h = {#"HOLEID": holecount ,
                             "HPID": hpid,
                             "STAGE": stage,
                             "CARD1": card_vals['card1'], "CARD2": card_vals['card2'],
                             "ACT": pf_act[1] ,
                             "CHIPAMT": pf_act[2]
                            }
                        #print(h)
                        holeC[holecount]=h

                    if (len(pf_act) > 3):
                        allin=1
                    else:
                        allin=0

                    hprank = next((val for key, val in card_ref.items() if key == hpid), None)
                    if stage == "FLOP" or stage == "TURN" or stage=="RIVER": #cards for flop, turn, river
                        ccount+=1
                        cards_info = {#"CID": ccount,
                                      "HPID": hpid,
                                      "STAGE": stage,
                                      "HANDRANK": hprank['handrank'],#card[1],
                                      "ACT": pf_act[1],
                                      "AllIn": allin,
                                      "CHIPAMT": pf_act[2]
                                }

                        cards[ccount]=cards_info

                    acount+=1
                    allin=0 #<---------------------------------reset all in boolean option
                elif record_action and line.startswith("FLOP:"):
                    stage = line.split(":")[0]
                    flop = line.split(":")[1].split(",")
                    card_ref.clear()

                elif record_action and line.startswith("CARDS:"):
                    card = line.split(":")[1].split(",")
                    acount+=1

                    hpid = next((key for key, val in hp_id.items() if val['PID'] == int(card[0])), None)

                    k={
                       "PID": int(card[0]),
                       "handrank": card[1]
                        }

                    card_ref[hpid]=k

                elif record_action and line.startswith("TURN:"):
                    stage = line.split(":")[0]
                    turn = line.split(":")[1]
                    card_ref.clear()

                elif record_action and line.startswith("RIVER:"):
                    stage = line.split(":")[0]
                    river = line.split(":")[1]
                    card_ref.clear()

                    board ={#"BID": bcount,
                            "FC1": flop[0], "FC2": flop[1], "FC3": flop[2], "TC": turn, "RC": river}

                    hand_board[bcount]=board

                elif record_action and line.startswith("WINNER:"):
                    hwcount+=1
                    winn = line.split(":")[1].split(",")

                    hpid = next((key for key, val in hp_id.items() if val['PID'] == int(winn[0])), None)
                    #cardRef = next((key for key, val in cards.items() if val['HPID'] == hpid), None)


                    win_info ={#"HEID": ecount,
                                "HPID":hpid,
                                #"PID": winn[0],
                                "POT": int(winn[1]),
                                #"CID": cardRef
                               }

                    hand_winners[hwcount] = win_info

                elif record_action and line.startswith("ELIMINATED:"):
                    elim = int(line.split(":")[1])
                    ecount+=1

                    #players eliminated at the end of the game
                    hpid = next((key for key, val in hp_id.items() if val['PID'] == int(elim)), None)

                    elim_info ={#"HEID": ecount,
                                "HPID":hpid
                               }

                    game_elim[ecount] = elim_info

                elif record_action and line.startswith("CONTINUES:"):
                    con = int(line.split(":")[1])
                    concount+=1

                    #players that continues from each table/game that moves to the next round
                    hpid = next((key for key, val in hp_id.items() if val['PID'] == int(con)), None)

                    con_info ={#"HCID": concount,
                               "HPID":hpid
                            }

                    game_con[concount]=con_info

                elif record_action and line.startswith("TOURNAMENTWINNER:"):
                    twcount+=1
                    twin = line.split(":")[1].split(",")

                    hpid = next((key for key, val in hp_id.items() if val['PID'] == int(twin[0])), None)

                    twin_info ={#"TWID": twcount,
                                #"HPID": hpid,
                                "TID":tid,
                                "PID": int(twin[0]),
                                "PAYOUT":float(twin[1]),
                               }

                    #if twin_info not in tour_winners:
                    tour_winners[twcount] = twin_info

**Parse Analysis and Import Process**


In [ ]:
%lprun -f ParseAndImport ParseAndImport()